# Regressão Logística

Regressão logística é uma técnica de classificação que pertence ao grupo dos classificadores lineares* e de alguma forma parecida com regressão linear. É uma técnica rápida e simples de entender, além de conveniente para interpretar os resultados. Apesar de ser, essencialmente, um método para classificação binária, pode ser usado para classificação multi-classes.

\* classificação baseada em uma combinação linear entre as variáveis de entrada, também chamado de _logit_.

ex: $f(X) = \alpha + b_1x_1 + b_2x_2 +\dots + b_nx_n$.

## Função Sigmoid

O objetivo da Regressão Logística é encontrar a função $p($**x**$)$ tal que as saídas estimadas $p(x_i)$ fiquem o mais próximo possível do valor real de $y_i$ para cada amostra $i = 1, 2, \dots, n$. Como, em um primeiro momento, vamos lidar exclusivamente de classificação binária, essas saídas devem ser apenas $0$ ou $1$. Sendo assim, é conveniente utilizar a função Sigmoid (caso especial da função logística).

<img src="assets/log-reg-1.webp" alt="sigmoid" width="400"/>


## Juntando a combinação linear com a função Sigmoid

A Regressão logística determina os melhores valores para o bias $\alpha$ e os pesos $b_1, b_2, \dots, b_n$, tal que a função $p($**x**$)$ se aproxime da classe real da amostra. O processo de encontrar os melhores valores para esses parâmetros é chamado de treinamento (ou _fitting_ ).

Nesse caso, **x** que a função Sigmoid ($p$) recebe é a saída da combinação linear $f(X) = \alpha + b_1x_1 + b_2x_2 +\dots + b_nx_n$.

## Como encontrar os melhores valores para o bias e os pesos?

Os melhores pesos são geralmente encontrados maximizando a função de verosimilhança logarítmica (_log-likelihood_ ) O método é chamado de maximização da verossimilhança (_maximum likelihood estimation_ - MLE) e é representado pela seguinte equação.

\begin{equation}
MLE = \sum_{i=1}^n(y_i log(p(x_i)) + (1 − y_i) log(1 − p(x_i))).
\end{equation}
<img src="assets/log-reg-4.webp" alt="log" width="400"/>

Os melhores parâmetros para maximizar essa equação podem ser encontrados usando técnicas matemáticas, essa parte será executada pelo scikit-learn.

## Implementação

### Exemplo 1 - Classificação binária com uma única variável de entrada

In [3]:
# Passo n° 1: Importando as bibliotecas necessárias
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [45]:
# Passo n° 2: Gerando dados para classificação

# Gerando os dados de entrada. Serão geradas 10 amostras com valores de 0 a 9
# note que os modelos do scikit-learn esperam um array composto com um array de características
# por amostra de entrada (mesmo que seja apenas 1 característica). Por isso fazemos um reshape.
x = np.arange(10).reshape(-1, 1) 

# gerando as classes para cada amostra
y = np.array([0, 0, 0, 0, 1, 1, 1, 1, 1, 1])
print('x = {}'.format(x))
print('y = {}'.format(y))

In [46]:
# Passo n° 3: Criando e treinando um modelo

# Criando ou estanciando o modelo:
#   solver é o algoritmo utilizado para otimizar os parâmetros.
#     liblinear é a melhor opção para datasets pequenos e binários.
#   random_state = 0, ou qualquer número, garante que a inicialização aleatória vai ser sempre igual, 
#     independente de quantas vezes rodar o algoritmo
model = LogisticRegression(solver='liblinear', random_state=0)

# O modelo possui diversos hiperparâmetros que podem ser ajustados, como penalidade (padrão L2), número 
#   de iterações, etc. Sugiro que deem uma olhada na documentação:
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

# Treinando o modelo
model.fit(x, y)

In [47]:
# Podemos extrair algumas informações do modelo, como por exemplo:

# as classes presentes no conjunto de dados
print('classes = {}'.format(model.classes_))

# o valor do bias
print('bias = {}'.format(model.intercept_))

# os pesos
print('pesos = {}'.format(model.coef_))

In [49]:
# Passo n° 4: Avaliando o modelo

# uma das formas de avaliar o modelo, é através a probabilidade  que ele dá à cada amostra
#   de pertencer a cada classe
probabilidades = model.predict_proba(x)

# uma outra forma mais comum é através da classe que o modelo estimou
classe_estimada = model.predict(x)

print('probabilidades = {}\n'.format(probabilidades))
print('classe_estimada = {}'.format(classe_estimada))

Podemos visualizar essa classificação na figura a seguir:
<img src="assets/log-reg-5.webp" alt="univariate" width="800"/>

Note que a classificação corresponde ao limiar $p(x) = 0.5$, que ocorre quando $f(x) = 0$. Esse valor é o limite da classificação entre as classe $0$ e $1$.

In [50]:
# uma outra métrica bem interessante é a acurácia, ou taxa de acerto do nosso modelo, 
#   que pode ser computada da seguinte maneira:

# esse valor representa a relação entre o número de acertos dividido pelo número total de amostras.
#   caso acerte todas as predições, o resultado é igual a 1.
model.score(x, y)

## Matriz de confusão

A matriz de confusão permite obter informações mais detalhadas sobre o resultado da classificação, levando em conta a taxa de acerto de cada classe. Nesse caso, utilizamos 4 novas métricas:

- Verdadeiros Negativos (True negatives - **TN**): amostras negativas (zeros) estimadas corretamente
- Verdadeiros Positivos (True positives - **TP**): amostras positivas (1's) estimadas corretamente
- Falsos Negativos (False negatives - **FN**): amostras negativas (zeros) estimadas incorretamente
- Falsos Positivos (False positives - **FP**): amostras positivas (1's) estimadas incorretamente

A matriz de confusão mostra esses valores da seguinte forma:

- **TN**: no canto superior esquerdo
- **FN**: no canto inferior esquerdo
- **FP**: no canto superior direito
- **TP**: no canto inferior direito

In [53]:
# podemos computar a matriz de confusão da seguinte maneira

# poderiamos usar model.predict(x), mas já temos os resultados das predições na variável classe_estimada
cm = confusion_matrix(y, classe_estimada)
print('CM = ')
print(cm)

A primeira linha da matriz de confusão mostra que 3 amostras da classe zero foram classificadas como classe zero, enquanto 1 amostra da classe zero foi classificada erroneamente como classe 1.

Do mesmo modo, a segunda linha mostra que nenhuma amostra da classe 1 foi classificada erroneamente como classe 0, pois todas as amostras (6) da classe 1 foram classificadas corretamente.

Podemos plotar a matriz de confusão para ter uma melhor visualização desses resultados:

In [54]:
fig, ax = plt.subplots(figsize=(8, 8))
ax.imshow(cm, cmap=plt.cm.Blues)
ax.grid(False)
ax.xaxis.set(ticks=(0, 1), ticklabels=('0s estimados', '1s estimados'))
ax.yaxis.set(ticks=(0, 1), ticklabels=('0s reais', '1s reais'))
ax.set_ylim(1.5, -0.5)
for i in range(2):
    for j in range(2):
        ax.text(j, i, cm[i, j], ha='center', va='center',  size=15)
plt.show()

Podemos também obter um relatório mais detalhado da classificação utilizando a função classification_report():

In [55]:
print(classification_report(y, classe_estimada))

Onde as métricas representam:

- **Precisão** = TP/TP+FP ou TN/TN+FN -> Taxa de acerto considerando todas as amostras classificadas como sendo da classe.
- **Recall** = TP/TP+FN ou TN/TN+FP -> Taxa de quantas amostras da classe foram corretamente classificadas.
- **F1, Macro AVG e Weighted AVG** -> Similar a acurácia, mas são ponderadas pelo número de amostras por classes.
- **support** -> número de amostras consideradas em cada caso

<img src="assets/prec_rec.png" alt="prec_rec" width="400"/>

## Melhorando nosso modelo

Podemos melhorar o modelo alterando alguns dos hiper-parâmetros. Podemos, por exemplo, alterar o valor do hiper-parâmetro C (um valor de regularização) de $1.0$ (padrão) para $10.0$:

In [29]:
model = LogisticRegression(solver='liblinear', C=10.0, random_state=0).fit(x, y)

Essa mudança já é capaz de gerar diferentes probabilidades:

In [56]:
print('bias = ',model.intercept_)
print('pesos = ',model.coef_)
print('probabilidades = ',model.predict_proba(x))
print('classes estimadas = ',model.predict(x))

Essa regularização gerou valores mais significativos nos bias e nos pesos, implicando em uma mudança na nossa função logit $f(x)$. Essa mudança também foi importante para distorcer a nossa função sigmoid, sendo que agora o valor de limiar ($p(x)=0.5$) foi deslocado mais para a direita, como mostrado na figura a seguir:

<img src="assets/log-reg-7.webp" alt="regularized" width="600"/>

printando novamente nosso relatório, podemos observar que dessa vez obtemos o valor máximo em todas as métricas, indicando que a classificação foi $100%$ satisfatória.

In [57]:
print('acurácia = ', model.score(x, y))
print('matriz de confusão = ')
print( confusion_matrix(y, model.predict(x)))
print(classification_report(y, model.predict(x)))

### Exemplo 2 - Similar ao exemplo 1, mudando apenas a classe da segunda amostra

In [37]:
# Passo 1: importando os pacotes e classes
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Passo 2: Gerando os dados
x = np.arange(10).reshape(-1, 1)
y = np.array([0, 1, 0, 0, 1, 1, 1, 1, 1, 1])

# Passo 3: Criando e treinando o modelo
model = LogisticRegression(solver='liblinear', C=10.0, random_state=0)
model.fit(x, y)

# Passo 4: avaliando o modelo
p_pred = model.predict_proba(x)
y_pred = model.predict(x)
score_ = model.score(x, y)
conf_m = confusion_matrix(y, y_pred)
report = classification_report(y, y_pred)

visualizando os resultados:

In [58]:
print('x:', x, sep='\n')
print('y:', y, sep='\n', end='\n\n')
print('bias:', model.intercept_)
print('pesos:', model.coef_, end='\n\n')
print('probabilidades:', p_pred, sep='\n', end='\n\n')
print('classes estimadas:', y_pred, end='\n\n')
print('acurácia:', score_, end='\n\n')
print('matriz de confusão:', conf_m, sep='\n', end='\n\n')
print('relatório:', report, sep='\n')

Nesse caso, 2 amostras foram classificadas erroneamente, como mostrado na figura a seguir:

<img src="assets/log-reg-8.webp" alt="regularized" width="600"/>

A figura acima revela uma característica importante da regressão logística: Ela não resolve problemas que não sejam linearmente separáveis. Em outras palavras, não é possível encontrar uma reta que separe as amostras, nesse caso, de forma a conseguir $100\%$ de acurácia.

### Exemplo 3 - Reconhecendo Dígitos

O proximo exemplo lida com uma abordagem de problema do mundo real, de reconhecimento de imagem, com multiplas classes. Para isso, utilizaremos um dataset composto por 1797 imagens de digitos de $8\times8$ pixels, representados por um vetor de $64$ posições.

In [14]:
# Passo 1: importando os pacotes

import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import load_digits
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [59]:
# Passo 2a: importando os dados

x, y = load_digits(return_X_y=True)

print('Dimensões x:', x.shape)
print('Dimensões y:', y.shape)
print('x=', x, sep='\n')
print('y=', y, sep='\n')

In [60]:
# Visualizando alguns digitos
sample = np.random.randint(0, len(x))
img = x[sample].reshape((8,8)).astype(np.uint8)

imgplot = plt.imshow(img, cmap="gray")

In [9]:
# Passo 2b: dividindo os dados em conjunto de treino e conjunto de teste

# Conjunto de treino é usado para ajustar o modelo (encontrar o bias e os pesos), 
#    enquanto o conjunto de testes é usado para avaliar o quão bom o modelo é para predizer casos
#    desconhecido, ou seja, amostras que não foram utilizadas no treinamento.

# Essa divisão é geralmente feita de modo aleatório, sendo que, geralmente, 70% a 80% dos dados é usado pra
#    treinamento e o restante para teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [10]:
# Passo 2c: Normalizando os dados

# A maioria dos algoritmos de aprendizado de máquina funcionam melhor com dados normalizados, ou seja, 
#   transformando os dados de modo que todas as características (features) tenham média zero e desvio
#   padrão igual a 1.

# Para normalizar os dados, seguimos 3 passos:
#   1 - computar a média e o desvio padrão de cada coluna de x (feature)
#   2 - subtrair o valor de cada característica pela média de sua coluna
#   3 - dividir essa diferença pelo desvio padrão da coluna.

# O processo pode ser feito usando a função .fit_transform() do pacote 
#    sklearn.preprocessing.StandardScaler:

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)

In [61]:
# Passo 3: Criando e treinando o modelo

# Processo bem parecido com o anterior. Note que selecionamos 'ovr' para o hiper-parâmetro 
#   multi_class. O objetivo é fazer a abordagem de classificação binária para cada classe (one-versus-all).
model = LogisticRegression(solver='liblinear', C=0.05, multi_class='ovr',
                           random_state=0)
model.fit(x_train, y_train)

In [32]:
# Passo 4: Avaliando o modelo

# A avaliação do modelo é similar a que fizemos nos exemplos anteriores. A diferença é que usaremos
#   o conjunto de teste para esse processo, em vez do conjunto de treinamento.

# primeiramente, precisamos normalizar as amostras do conjunto de testes:
x_test = scaler.transform(x_test)

# em seguida, chamamos a função predict, passando o conjunto de testes:
y_pred = model.predict(x_test)

In [62]:
# podemos comparar o quanto o modelo acertou nos dados de treinamento e teste, e assim descobrir se o 
#   modelo não decorou o conjunto de testes (overfit) em vez de aprender o padrão dos dados
print('Acurácia sobre conjunto de treinamento = ',model.score(x_train, y_train))
print('Acurácia sobre conjunto de teste = ',model.score(x_test, y_test))

In [63]:
# De modo similar, podemos gerar a matriz de confusão:

# A matriz permite ver em quais classes o modelo está confundindo e com quais classes.
confusion_matrix(y_test, y_pred)

In [64]:
# plotando a matriz

cm = confusion_matrix(y_test, y_pred)

fig, ax = plt.subplots(figsize=(8, 8))
ax.imshow(cm, cmap=plt.cm.Blues)
ax.grid(False)
ax.set_xlabel('Valores estimados', fontsize=15, color='black')
ax.set_ylabel('Valores reais', fontsize=15, color='black')
ax.xaxis.set(ticks=range(10))
ax.yaxis.set(ticks=range(10))
ax.set_ylim(9.5, -0.5)
for i in range(10):
    for j in range(10):
        ax.text(j, i, cm[i, j], ha='center', va='center')
plt.show()

In [65]:
# Como vimos anteriormente, podemos também gerar um relatório com os resultados:

print(classification_report(y_test, y_pred))

# Exercícios:

1. Reproduzir o último exercício trocando o dataset load_digits pelo dataset MNIST. Utilize apenas as primeiras 2 mil amostras para treinamento e as próximas mil amostras para teste.

Material baseado em: https://realpython.com/logistic-regression-python/